In [ ]:
import ibis

import ibis.selectors as s
import plotly.express as px

from python_analytics_accelerator.metrics import (
    pulls_t,
    stars_t,
    forks_t,
    issues_t,
    commits_t,
    downloads_t,
)

ibis.options.interactive = True
ibis.options.repr.interactive.max_rows = 5
ibis.options.repr.interactive.max_columns = None

px.defaults.template = "plotly_dark"

In [ ]:
pulls_t

In [ ]:
stars_t

In [ ]:
forks_t

In [ ]:
issues_t

In [ ]:
commits_t

In [ ]:
downloads_t